### Install modules

In [0]:
!pip install mlflow langchain databricks-vectorsearch databricks-sdk mlflow[databricks] dotenv langchain_community langchain-core langgraph

In [0]:
!pip install IPython

In [0]:
!pip install gradio

In [0]:
!pip install streamlit

In [0]:
%python
dbutils.library.restartPython()

### Import modules

In [0]:
import requests

In [0]:
import os
from dotenv import load_dotenv
import IPython
import datetime

In [0]:

# Langraph components ---------
from typing import Annotated, List, Dict, Any
from typing_extensions import TypedDict
from langgraph.graph import START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.messages import AnyMessage
from langgraph.graph import StateGraph
from IPython.display import Image, display
from langgraph.graph.state import CompiledStateGraph
from langgraph.checkpoint.memory import MemorySaver

memory: MemorySaver = MemorySaver()




# Databricks models and embeddings -------------
from langchain_community.chat_models import ChatDatabricks
from langchain_community.embeddings import DatabricksEmbeddings

# vector search ----------
from databricks.vector_search.client import VectorSearchClient
from langchain_community.vectorstores import DatabricksVectorSearch

# MLFLow -------------
from mlflow.models import infer_signature
import mlflow
import langchain
import langchain_core
import langchain_community
from langchain_core.runnables import Runnable
from langchain_core.messages import HumanMessage
from langgraph.graph import START, END 


# UI
import gradio as gr

### Set env

In [0]:
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
os.environ["DATABRICKS_HOST"] = host


In [0]:
load_dotenv()

### Databricks model and embeddings

In [0]:
chat_model = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 1024)

In [0]:
embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

### Create Vector DB retriever function

In [0]:
index_name="analytics.models.docs_idxs"
VECTOR_SEARCH_ENDPOINT_NAME="document_vector_endpoint"

In [0]:

# --- vectorestore retriver 
def get_retriever(persist_dir: str = None):
    os.environ["DATABRICKS_HOST"] = host
    #Get the vector search index
    vsc = VectorSearchClient(workspace_url=host, personal_access_token=os.environ["DATABRICKS_TOKEN"])
    vs_index = vsc.get_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        index_name=index_name
    )

    # Create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index, text_column="text", embedding=embedding_model
    )
    return vectorstore.as_retriever()


In [0]:
# --- web search tool
def web_search_tool(query: str) -> str:
    headers = {
    "X-API-KEY": os.environ["SERPAPI_KEY"],
    "Content-Type": "application/json"
        }

    params = {
    "q": query
    }

    res = requests.post("https://google.serper.dev/search", json=params, headers=headers)
    res_json = res.json()
    results = res_json.get("organic", [])
  
    if results:
        return results[0].get("snippet", "")
    return "No useful web data found."



### Compile Langgraph

In [0]:
retriever = get_retriever()

In [0]:





# --- Langgraph State Definition
class State(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    retrieved_context: str
    initial_chatbot_response: str

# --- Langgraph Graph Builder
graph_builder = StateGraph(State)
retriever = get_retriever()


def retrieval_node(state: State):

    print("Executing conversational retrieval...")
    conversation_for_retrieval = "\n".join([msg.content for msg in state['messages']])
    if not conversation_for_retrieval.strip():
        return {"retrieved_context": "No query provided."}
    docs = retriever.get_relevant_documents(conversation_for_retrieval)
    if not docs:
        context = "No relevant HR policy document was found for the query."
    else:
        context = "\n\n".join([doc.page_content for doc in docs])
    return {"retrieved_context": context}


def chatbot_node(state: State):

    prompt = f"""You are an HR Assistant chatbot designed to provide accurate and up-to-date answers to employees HR-related queries. Your responses should be clear, concise, and directly based on the company's official policies. If the question is not related to one of these topics, kindly decline to answer. If you don't know the answer, say that you don't know; don't try to make up an HR policy answer. Keep the answer as concise as possible. Provide all answers only in English. Use the following pieces of context to answer the HR Policy question:
    {state['retrieved_context']}"""

    llm_messages = [SystemMessage(content=prompt)]

    llm_messages.extend(state['messages']) # Adds all messages from the current conversation

    response = chat_model.invoke(llm_messages)
    return {"messages": [response],"initial_chatbot_response": response.content}
    
 

def enhancer_node(state: State):
 
    user_query = [msg for msg in state["messages"] if isinstance(msg, HumanMessage)][-1].content
    initial_answer = state["initial_chatbot_response"]
    retrieved_hr_context = state["retrieved_context"]
    current_date_str = datetime.date.today().strftime("%B %d, %Y")

    web_context = web_search_tool(user_query)
    print(f"Web search conducted. Results: {web_context[:100]}...")

    synthesis_prompt = f"""
    You are an HR Assistant chatbot providing comprehensive and accurate answers.
    You have already generated an initial response based on internal HR policies.
    Now, you have additional external information from a web search.

    Your task is to **synthesize** all available information to provide the best possible answer to the user.
    **Do NOT simply copy-paste directly from the web search results.**
    Integrate the web search results with the initial HR policy response, making sure to:
    - Add any missing factual details (like specific dates or external holiday names if relevant and not in HR policy).
    - Clarify or correct information if the web search provides more accurate external facts.
    - Prioritize information from the **internal HR policy** for HR-related questions. Use web search primarily for external facts (e.g., current date-specific holidays, general knowledge).
    - If the web search provides no new relevant information, you can mostly stick to the initial answer, perhaps rephrasing slightly for clarity.
    - If the web search contradicts internal HR policy on a *policy* matter, state that the company policy is the primary source.
    - Maintain a neutral, respectful, and concise tone.
    - Provide all answers only in English.

    --- User Query ---
    {user_query}

    --- Initial HR Policy Response ---
    {initial_answer}

    --- Internal HR Policy Context (from retrieval) ---
    {retrieved_hr_context}

    --- External Web Search Results (if available) ---
    {web_context}

    --- Current Date ---
    {current_date_str}

    Based on the above, provide the final, synthesized answer:
    """
    llm_messages_for_llm = [SystemMessage(content=synthesis_prompt)]
    

    last_human_message = next((msg for msg in reversed(state['messages']) if isinstance(msg, HumanMessage)), None)
    if last_human_message:
        llm_messages_for_llm.append(last_human_message)
    else:
        print("Warning: No human message found in state for enhanced_response_node.")
        
    final_response = chat_model.invoke(llm_messages_for_llm)
    
    # LangGraph's 'add_messages' will correctly append this new AIMessage to state['messages']
    # This also effectively replaces the 'initial_chatbot_response' in the conversation history
    return {"messages": [final_response]}


# --- The Router Node ---
def should_enhance(state: State) -> str:
    """
    Uses an LLM to decide if the user's query requires a web search for external facts.
    """
    
    user_query = [msg.content for msg in state["messages"] if isinstance(msg, HumanMessage)][-1]
    
    # Use an LLM to make the routing decision
    routing_prompt = f"""You are a routing agent. Your goal is to decide if a user's query requires a web search for external, factual information (like specific dates, holidays, states, current events) or if it can be answered solely by an internal HR policy document.

    User Query: "{user_query}"

    Does this query likely require a web search for up-to-date, external facts? Answer only with the single word 'yes' or 'no'.
    """
    response = chat_model.invoke(routing_prompt)
    decision = response.content.strip().lower()
    print(f"--- [Router] LLM decision: '{decision}' ---")

    if 'yes' in decision:
        print("--- [Router] Path chosen: enhance ---")
        return "enhance"
    else:
        print("--- [Router] Path chosen: end ---")
        return "end"

# --- Add Nodes to the Graph ---
graph_builder.add_node("retrieval_node", retrieval_node)
graph_builder.add_node("chatbot_node", chatbot_node)
graph_builder.add_node("enhancer_node", enhancer_node)

# --- Define Graph Edges with Conditional Logic ---
graph_builder.add_edge(START, "retrieval_node")
graph_builder.add_edge("retrieval_node", "chatbot_node")

# After the chatbot node, we call the router to decide the next step
graph_builder.add_conditional_edges(
    "chatbot_node",
    should_enhance,
    {
        # If the router returns "enhance", we go to the enhancer_node.
        "enhance": "enhancer_node",
        # If the router returns "end", we finish the graph execution.
        "end": END
    }
)

graph_builder.add_edge("enhancer_node", END)

# --- Compile the Graph ---
graph = graph_builder.compile(checkpointer=memory)





In [0]:
#display(Image(graph.get_graph().draw_mermaid_png()))

In [0]:
graph

In [0]:
graph.nodes

In [0]:


from langchain_core.runnables import Runnable
import uuid
from typing import Any, Dict, Optional
from langchain_core.runnables import Runnable
from langchain_core.messages import AIMessage, HumanMessage

class LanggraphRunnable(Runnable):

    def invoke(self, input_text: str, config: Optional[Dict] = None) -> str:
        run_config = config or {}
        
        # Define the input payload for the graph
        input_payload = {'messages': [HumanMessage(content=input_text)]}

        final_state = graph.invoke(input_payload, config=run_config)
        
        # Safely extract the last message from the final state dictionary
        if final_state and "messages" in final_state:
            if final_state["messages"]:
                last_message = final_state["messages"][-1]
                if isinstance(last_message, AIMessage):
                    return last_message.content

        # This will be returned only if something goes wrong
        return "Error: Could not extract a valid response from the chatbot."

langraph_runnable = LanggraphRunnable()

In [0]:
question = "how do i do expenses when I travel?"
current_conversation_id = str(uuid.uuid4())

run_config = {"configurable": {"thread_id": current_conversation_id}}
answer = langraph_runnable.invoke(question, config=run_config)


if answer:
    print("Chatbot Response:", answer)
else:
    print("No response from the chatbot.")


In [0]:
question = "how do I do expenses when I travel?"
answer = langraph_runnable.invoke(question,  config=run_config)

if answer:
    print("Chatbot Response:", answer)
else:
    print("No response from the chatbot.")


In [0]:


question = "How do I do expenses when I travel?"
answer = langraph_runnable.invoke(question, config=run_config)

if answer:
    print("Chatbot Response:", answer)
else:
    print("No response from the chatbot.")


In [0]:


question = "What are the key takeaways of harrassment policy?"
answer = langraph_runnable.invoke(question, config=run_config)

if answer:
    print("Chatbot Response:", answer)
else:
    print("No response from the chatbot.")


### Simple Gradio UI

In [0]:


def predict(question):
    answer = langraph_runnable.invoke(question)
    return answer if answer else "No response from the chatbot."


demo = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(label="Ask your HR question"),
    outputs=gr.Textbox(label="HR Assistant Response"),
    title="HR Assistant Chatbot",
    description="Ask any HR-related question and I will try my best to answer based on company policies."
)


if __name__ == "__main__":
    demo.launch(share=True)

## Streamlit UI with Conversation History

In [0]:

import streamlit as st
import requests

## langgraph
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from langchain_core.runnables import Runnable

## mlflow
import mlflow


loaded_model = graph



In [0]:
class LanggraphRunnable(Runnable):
    def __init__(self, user_name: str = None):
        self.user_name = user_name

    def invoke(self, input: str, history: list[dict] = None):
        message_objects = []

        if history:
            for msg in history:
                if msg["role"] == "user":
                    content = msg["content"]
                    if self.user_name:
                        content = f"Please remember, my name is {self.user_name}. " + content
                    message_objects.append(HumanMessage(content=content))
                elif msg["role"] == "assistant":
                    message_objects.append(AIMessage(content=msg["content"]))

        # Inject instruction into current input
        if self.user_name:
            input = f"Please remember, my name is {self.user_name}. " + input

        message_objects.append(HumanMessage(content=input))

        # Call the graph
        final_output = None
        for output in loaded_model.stream({'messages': message_objects}):
            final_output = output

        if final_output and "enhancer_node" in final_output and "messages" in final_output["enhancer_node"]:
            messages = final_output["enhancer_node"]["messages"]
            for msg in messages:
                if isinstance(msg, AIMessage):
                    return msg.content
        return "No response from the chatbot."


In [0]:
chatbot = LanggraphRunnable(user_name="Bob")

history = [
    {"role": "user", "content": "Hi I am Bob, please mention my name before answering all questions. What is the official work-from-home policy?"},
    {"role": "assistant", "content": "Hi Bob, Employees may work from home up to 3 days per week with manager approval."}
]

user_input = "Can employees also work from home?"
response = chatbot.invoke(user_input, history=history)
print("🤖 Chatbot:", response)


In [0]:

user_input = "How do I do expenses when I travel?"
response = chatbot.invoke(user_input, history=history)
print("🤖 Chatbot:", response)
